<a href="https://colab.research.google.com/github/NeuralProcess/HW7/blob/master/LRUC_final%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sympy import Symbol, solve

Cost = pd.read_excel('/Problem Data.xlsx', sheet_name = 'Cost and Heat Rate Data')
Load = pd.read_excel('/Problem Data.xlsx', sheet_name = 'Load Forcast Data')
Start_Shutdown = pd.read_excel('/Problem Data.xlsx', sheet_name = 'Startup and shutdown data')

def Cost_Gen(a, b, c, d, x):
  y =  a + (b * x) + (c * (x ** 2)) + (d * (x ** 3))
  return y

def IC_Gen(b, c, d, x):
  y = b + (2*c * x) + (3*d * (x ** 2))
  return y

In [71]:
Cost_Max_Gen = np.zeros((10, 1))
Gen_Max = np.zeros((10, 1))
Gen_Min = np.zeros((10, 1))
Gen_Num = np.zeros((10, 1))
A = np.zeros((10, 1))
B = np.zeros((10, 1))
C = np.zeros((10, 1))
D = np.zeros((10, 1))
IC_Max_Gen = np.zeros((10, 1))
Min_Up_Time = np.zeros((10, 1))
Min_Down_Time = np.zeros((10, 1))
Start_Up_Cost = np.zeros((10, 1))
Status = np.zeros((10, 1))
Status_Time = np.zeros((10, 1))

for i in range(1,11):
  Cost_Max_Gen_i = Cost_Gen(Cost.loc[i-1]["A"], Cost.loc[i-1]["B"], Cost.loc[i-1]["C"], Cost.loc[i-1]["D"], Cost.loc[i-1]["Max MW"])
  Cost_Max_Gen[i-1] = Cost_Max_Gen_i
  Gen_Num[i-1] = i
  Gen_Max[i-1] = Cost.loc[i-1]["Max MW"]
  Gen_Min[i-1] = Cost.loc[i-1]["Min MW"]
  A[i-1] = Cost.loc[i-1]["A"]
  B[i-1] = Cost.loc[i-1]["B"]
  C[i-1] = Cost.loc[i-1]["C"]
  D[i-1] = Cost.loc[i-1]["D"]
  IC_Max_Gen_i = IC_Gen(Cost.loc[i-1]["B"], Cost.loc[i-1]["C"], Cost.loc[i-1]["D"], Cost.loc[i-1]["Max MW"])
  IC_Max_Gen[i-1] = IC_Max_Gen_i
  Min_Up_Time[i-1] = Start_Shutdown.loc[i-1]["Min Up Time"]
  Min_Down_Time[i-1] = Start_Shutdown.loc[i-1]["Min Down Time"]
  Start_Up_Cost[i-1] = Start_Shutdown.loc[i-1]["Startup Cost"]
  Status[i-1] = Start_Shutdown.loc[i-1]["Status"]
  Status_Time[i-1] = Start_Shutdown.loc[i-1]["Status time"]


In [72]:
Load_MW = np.zeros((30, 1))
Load_Hr = np.zeros((30, 1))

for i in range(1, 25):
  Load_MW[i-1] = Load.loc[i-1]["Load"]
  Load_Hr[i-1] = Load.loc[i-1]["Hours"]

In [73]:
x = Symbol('x')

Dual_Min_G = np.zeros((24, 10))
U_G = np.zeros((24, 10))
LR_J = np.zeros((24, 10))
Lambda = np.zeros((24, 1))
q = 0
Stopping_Criterion = np.zeros((100, 1))
Stopping_Criterion[0, 0] = 50
P_q = np.zeros((24, 10))
P = np.zeros((24, 10))
U_Sum = np.zeros((24, 1))

for k in range(0, 100):
  
  if Stopping_Criterion[k, 0] <= 0.05:
    break

  else:
    for t in range(0, 24):
      for i in range(0, 2):
        if B[i, 0] < Lambda[t, 0]:
          P_q[t, i] = Gen_Max[i, 0]
        else:
          P_q[t, i] = 0
      for i in range(3, 10):
        if ((4*C[i, 0])**2 - (12*D[i, 0]) * (B[i, 0] - Lambda[t, 0])) > 0:
          if (-2 * C[i, 0] + ((4 * C[i, 0]**2) - 12*D[i, 0] * (B[i, 0] - Lambda[t, 0]))**0.5) / (6 * D[i, 0]) > Gen_Max[i, 0]:
            P_q[t, i] = Gen_Min[i, 0]
          elif (-2 * C[i, 0] + ((4 * C[i, 0]**2) - 12*D[i, 0] * (B[i, 0] - Lambda[t, 0]))**0.5) / (6 * D[i, 0]) < Gen_Min[i, 0]:
            P_q[t, i] = 0
          else:
            P_q[t, i] = (-2 * C[i, 0] + (4 * C[i, 0]**2 - 12*D[i, 0] * (B[i, 0] - Lambda[t, 0]))**0.5) / (6 * D[i, 0])
        else:
          if (-2 * C[i, 0] + ((4 * C[i, 0]**2) - 12*D[i, 0] * (B[i, 0] - Lambda[t, 0]))**0.5) / (6 * D[i, 0]) < Gen_Min[i, 0]:
          P_q[t, i] = 0

    print(P_q)
    # C(P)가 양수면 발전기 끄기 음수면 발전기 켜기
    for i in range(0, 10):
      for j in range(0, 24):
        if P_q[j, i] > 0:
          U_G[j, i] = 1
        else:
          U_G[j, i] = 0    

    print(U_G)
    for t in range(0, 24):
      for i in range(0, 10):
        U_Sum[t, 0] = U_Sum[t, 0] + U_G[t, i]
    
    for t in range(0, 24):
      for i in range(0, 10):
        if U_Sum[t, 0] >= 1:
          equation = Cost_Gen(A[i, 0], (B[i, 0] - Lambda[t, 0]), C[i, 0], D[i, 0], P_q[t, i])
          Prim = equation * U_G[t, i]
          q = Prim + q

          if i <= 2:
            if B[i, 0] < Lambda[t, 0]:
              P[t, i] = Gen_Max[i, 0]
            else:
              P[t, i] = 0          

          else:
            if 4*C[i, 0]**2 - 12*D[i, 0] * B[i, 0] > 0:
              if (-2 * C[i, 0] + (4 * C[i, 0]**2 - 12*D[i, 0] * (B[i, 0]))**0.5) / (6 * D[i, 0]) > Gen_Max[i, 0]:
                P[t, i] = Gen_Min[i, 0]
              elif (-2 * C[i, 0] + (4 * C[i, 0]**2 - 12*D[i, 0] * (B[i, 0]))**0.5) / (6 * D[i, 0]) < Gen_Min[i, 0]:
                P[t, i] = 0
              else:
                P[t, i] = (-2 * C[i, 0] + (4 * C[i, 0]**2 - 12*D[i, 0] * (B[i, 0]))**0.5) / (6 * D[i, 0])
            else:
              P[t, i] = 0
          J = J + Cost_Gen(A[i, 0], B[i, 0], C[i, 0], D[i, 0], P[t, i])

        else:
          q = 10
          P[t, i] = 0
          J = 100

    #89pge에 있는 w
    W = np.zeros((24, 1))
    for j in range(0, 24):
      for i in range(0, 10):
        W[j, 0] = Load_MW[j] - (P_q[j, i] * U_G[j, i])

    

    df = pd.DataFrame(P_q, index = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21', 'T22', 'T23', 'T24'],
                    columns = ['GEN1', 'GEN2', 'GEN3', 'GEN4', 'GEN5', 'GEN6', 'GEN7', 'GEN8', 'GEN9', 'GEN10'])
    print(df)



    for i in range(0, 24):
      if W[i, 0] >= 0:
        Lambda[i, 0] = Lambda[i, 0] + (W[i, 0] * 0.01)
      else:
        Lambda[i, 0] = Lambda[i, 0] + (W[i, 0] * 0.002)

    print(Lambda)

    Stopping_Criterion[k+1, 0] = (J - q) / q
    print(J)
    print(q)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars


In [26]:
RUN = solve(5*x**2 + 5*x + 5)
print(RUN)
max(RUN)

[-1/2 - sqrt(3)*I/2, -1/2 + sqrt(3)*I/2]


TypeError: ignored

In [ ]:

P_Test = np.zeros((24, 10))
IC_Gen_Solve_List = np.zeros((24, 24))

for t in range(1, 24):
  for i in range(1, 10):

    IC_Gen_Solve = np.array(solve(IC_Gen(B[i, 0], C[i, 0], D[i, 0], x)))

    if len(IC_Gen_Solve) == 0:
      IC_Gen_Solve_Arr = np.zeros((1, 1))
      IC_Gen_Solve_Arr[0] = 0
  
    if isinstance(IC_Gen_Solve_Arr[0], complex):
      IC_Gen_Solve_Min = 0
    else:
      if isinstance(IC_Gen_Solve, int):
        IC_Gen_Solve_Min = IC_Gen_Solve
      else:
        IC_Gen_Solve_Min = max(IC_Gen_Solve_Arr)
    

    if (IC_Gen_Solve_Min >= Gen_Min[i, 0]) & (IC_Gen_Solve_Min <= Gen_Max[i, 0]):
      if Cost_Gen(A[i, 0], B[i, 0], C[i, 0], D[i, 0], IC_Gen_Solve_Min) >= 0:
        P_Test[t, i] = 0
      else:
        P_Test[t, i] = IC_Gen_Solve_Min
    elif IC_Gen_Solve_Min <= Gen_Min[i, 0]:
      P_Test[t, i] = 0
    elif IC_Gen_Solve_Min >= Gen_Max[i, 0]:
      P_Test[t, i] = Gen_Max[i, 0]



print(P_Test)
  

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
# iteration 1 // Generation 4

P_Test = np.zeros((24, 10))
IC_Gen_Solve_List = np.zeros((24, 24))

# i 는 발전기 숫자, t는 시간
# 행이 시간 열이 발전기

for i in range(1, 10):

  IC_Gen_Solve = np.array(solve(IC_Gen(B[i, 0], C[i, 0], D[i, 0], x)))

  if len(IC_Gen_Solve) == 0:
    IC_Gen_Solve = 0

print(IC_Gen_Solve)

  


[-9.80952380952381 - 131.893277822237*I
 -9.80952380952381 + 131.893277822237*I]


In [ ]:

  if IC_Gen_Solve[i, 0] == 0:
    IC_Gen_Solve_Min = 0
  elif (IC_Gen_Solve[i, 0].is_real):
    IC_Gen_Solve_Min = max(IC_Gen_Solve)
  else:
    IC_Gen_Solve_Min = 0

  print(IC_Gen_Solve_Min)

  if (IC_Gen_Solve_Min >= Gen_Min[i, 0]) & (IC_Gen_Solve_Min <= Gen_Max[i, 0]):
    if Cost_Gen(A[i, 0], B[i, 0], C[i, 0], D[i, 0], IC_Gen_Solve_Min) >= 0:
      P_Test[i, 0] = 0
    else:
      P_Test[i, 0] = IC_Gen_Solve_Min
  elif IC_Gen_Solve_Min <= Gen_Min[i, 0]:
    P_Test[i, 0] = 0
  elif IC_Gen_Solve_Min >= Gen_Max[i, 0]:
    P_Test[i, 0] = Gen_Max[i, 0]


sympy.core.add.Add

In [ ]:
# iteration 1 // Generation 1


In [ ]:
# Iteration 2 // Generation all, lambda = Lambda
#find P

P = np.zeros((24, 10)) # 발전값

# 람다값 0넣을지 말지
for j in range(0, 10):
  for i in range(0, 24):
    Sub_Diff = B[j, 0] - Lambda[i, 0]
    if Sub_Diff < 0:
      P[i, j] = (Lambda[i, 0] - B[j, 0])
    else:
      P[i, j] = 0


df = pd.DataFrame(P, index = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21', 'T22', 'T23', 'T24'],
                  columns = ['GEN1', 'GEN2', 'GEN3', 'GEN4', 'GEN5', 'GEN6', 'GEN7', 'GEN8', 'GEN9', 'GEN10'])
print(df)

# 발전량 양수면 발전기 켜기 On,Off
for i in range(0, 10):
  for j in range(0, 24):
    if P[j, i] > 0:
      Dual_u_G[j, i] = 1
    else:
      Dual_u_G[j, i] = 0

print(Dual_u_G)

     GEN1          GEN2  GEN3   GEN4   GEN5   GEN6   GEN7  GEN8  GEN9  GEN10
T1    0.0  0.000000e+00   0.0  0.000  0.000  0.000  0.000  0.00  0.00   0.00
T2    0.0  0.000000e+00   0.0  0.000  0.000  0.000  0.000  0.00  0.00   0.00
T3    0.0  0.000000e+00   0.0  0.000  0.000  0.000  0.000  0.00  0.00   0.00
T4    0.6  1.776357e-15   0.0  5.127  5.127  5.127  5.127  4.07  4.07   3.18
T5    0.0  0.000000e+00   0.0  0.000  0.000  0.000  0.000  0.00  0.00   0.00
T6    2.4  1.800000e+00   0.0  6.927  6.927  6.927  6.927  5.87  5.87   4.98
T7    0.0  0.000000e+00   0.0  0.000  0.000  0.000  0.000  0.00  0.00   0.00
T8    0.0  0.000000e+00   0.0  0.000  0.000  0.000  0.000  0.00  0.00   0.00
T9    0.0  0.000000e+00   0.0  0.000  0.000  0.000  0.000  0.00  0.00   0.00
T10   0.0  0.000000e+00   0.0  0.000  0.000  0.000  0.000  0.00  0.00   0.00
T11   0.0  0.000000e+00   0.0  0.000  0.000  0.000  0.000  0.00  0.00   0.00
T12   0.0  0.000000e+00   0.0  0.000  0.000  0.000  0.000  0.00  0.00   0.00

In [ ]:
# equation은 각각의 발전기 subproblem
# Prim 은 일단 넘어가
for j in range(0, 10):
  for i in range(0, 24):
    equation = IC_Gen(B[j, 0], C[j, 0], D[j, 0], P[i, j])
    Prim = equation * Dual_u_G[i, j]
    q = Prim + q

print(Dual_u_G)
print(q)

#89page에 있는 w
W = np.zeros((24, 1))
for j in range(0, 24):
  for i in range(0, 10):
    W[j, 0] = Load_MW[j] - (P[j, i] * Dual_u_G[j, i])

print(W)

# time 1에서 L값 구하기
LR = np.zeros((24, 1))
LR_J = np.zeros((24, 1))
for j in range(0, 24):
  for i in range(0, 10):
    LR[j, 0] = LR[j, 0] + (Cost_Gen(A[i, 0], B[i, 0], C[i, 0], D[i, 0], P[j, i]) * Dual_u_G[j, i])
  LR_J[j, 0] = 10000
print(LR)
print(LR_J)

Total_J = sum(LR_J)
print(Total_J)


for i in range(0, 24):
  if W[i, 0] >= 0.05:
    Lambda[i, 0] = Lambda[i, 0] + (W[i, 0] * 0.01)
  else:
    Lambda[i, 0] = Lambda[i, 0] + (W[i, 0] * 0.002)

print(Lambda)

#Stopping_Criterion = (Total_J - q) / q
#while Stopping_Criterion > 0.05:
#  k = k + 1


#  if Stopping_Criterion <= 0.05:
#    break






[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
507.1731832732242
[[ 482.  ]
 [ 470.  ]
 [ 805.  ]
 [1416.82]
 [ 640.  ]
 [1595.02]
 [ 880.  ]
 [ 790.  ]
 [ 560.  ]
 [ 560.  ]
 [ 570.  ]
 [ 830.  ]
 [ 880.  ]
 [ 780.  ]
 [ 800.  ]
 [ 720.  ]
 [ 480.  ]
 [

NameError: ignored

In [ ]:
x = Symbol('x')

Dual_Min_G = np.zeros((24, 10))
Dual_u_G = np.zeros((24, 10))
LR_J = np.zeros((24, 10))
Lambda = np.zeros((24, 1))
q = 0
Stopping_Criterion = np.zeros((100, 1))
Stopping_Criterion[0, 0] = 50
P = np.zeros((24, 10))

for k in range(0, 100):
  
  if Stopping_Criterion[k, 0] <= 0.05:
    break
  else:

    # 여기서 람다값할때 ...... C(P1) 의 발전기 값이 양이되면 꺼지는거 자너
    # 초기의 람다 값은 0 → 람다값은 계속 업데이트 되는거 안구해도됨
    # 그러면 우리는 C(P1)가 최소가되는 P1의 값을 찾아야해
    # 그래서 그 P1을 계속 써먹어야지?
    

    for j in range(0, 10):
      for i in range(0, 24):
        Sub_Diff = B[j, 0] - Lambda[i, 0]
        if Sub_Diff < 0:
          P[i, j] = (Lambda[i, 0] - B[j, 0])
        else:
          P[i, j] = 0


    df = pd.DataFrame(P, index = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'T17', 'T18', 'T19', 'T20', 'T21', 'T22', 'T23', 'T24'],
                    columns = ['GEN1', 'GEN2', 'GEN3', 'GEN4', 'GEN5', 'GEN6', 'GEN7', 'GEN8', 'GEN9', 'GEN10'])
    print(df)

    # C(P)가 양수면 발전기 끄기 음수면 발전기 켜기
    for i in range(0, 10):
      for j in range(0, 24):
        if P[j, i] > 0:
          Dual_u_G[j, i] = 1
        else:
          Dual_u_G[j, i] = 0

    print(Dual_u_G)

    for j in range(0, 10):
      for i in range(0, 24):
        equation = IC_Gen(B[j, 0], C[j, 0], D[j, 0], P[i, j])
        Prim = equation * Dual_u_G[i, j]
        q = Prim + q

    print(Dual_u_G)
    print(q)

    #89page에 있는 w
    W = np.zeros((24, 1))
    for j in range(0, 24):
      for i in range(0, 10):
        W[j, 0] = Load_MW[j] - (P[j, i] * Dual_u_G[j, i])

    print(W)

    # time 1에서 L값 구하기
    LR = np.zeros((24, 1))
    LR_J = np.zeros((24, 1))
    for j in range(0, 24):
      for i in range(0, 10):
        LR[j, 0] = LR[j, 0] + (Cost_Gen(A[i, 0], B[i, 0], C[i, 0], D[i, 0], P[j, i]) * Dual_u_G[j, i])
      LR_J[j, 0] = 10000
    print(LR)
    print(LR_J)

    Total_J = sum(LR_J)
    print(Total_J)


    for i in range(0, 24):
      if W[i, 0] >= 0:
        Lambda[i, 0] = Lambda[i, 0] + (W[i, 0] * 0.01)
      else:
        Lambda[i, 0] = Lambda[i, 0] + (W[i, 0] * 0.002)

    print(Lambda)

    Stopping_Criterion[k+1, 0] = (Total_J - q) / q

     GEN1  GEN2  GEN3  GEN4  GEN5  GEN6  GEN7  GEN8  GEN9  GEN10
T1    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T2    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T3    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T4    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T5    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T6    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T7    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T8    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T9    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T10   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T11   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T12   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T13   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T14   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
T15   0.0   0.0   0.0   0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:82: RuntimeWarning: divide by zero encountered in true_divide


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 [1004.08408084]
 [ 594.01675844]]
[[15976.70228931]
 [15555.28175811]
 [28567.21465932]
 [62320.56877734]
 [21809.38749891]
 [75534.16762441]
 [31905.33485188]
 [27920.71998824]
 [18787.27978817]
 [18787.27978817]
 [19156.89188533]
 [29660.0730069 ]
 [31905.33485188]
 [27493.4943096 ]
 [28350.9565988 ]
 [24991.4602918 ]
 [15906.27085042]
 [15730.53394587]
 [28350.9565988 ]
 [63702.6169973 ]
 [38850.04552643]
 [75534.16762441]
 [58297.07682895]
 [28350.9565988 ]]
[[10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]
 [10000.]]
[240000.]
[[135.45332254]
 [132.15040571]
 [224.31887732]
 [393.47793981]
 [178.9284277 ]
 [442.98147933]
 [244.95089988]
 [220.19247281]
 [156.92093697]
 [156.92093697]
 [159.67187331]
 [231.19621817]
 [244.95089988]
 [217.44153647]
 [222.94340915]
 [200